# 7장. 합성곱 신경망(CNN)

https://github.com/WegraLee/deep-learning-from-scratch

이 코드의 내용은 Deep Learning from Scratch를 참고했음을 밝힙니다.

### 전체 구조(p227)

* 합성곱 계층(convolutional layer) + 풀링 계층(pooling layer)
* 완전연결(fully-connected, 전결합) : 인접하는 계층의 모든 뉴런과 결합
* Affine 계층 : 완전히 연결된 계층

* 완전연결 계층(Affine 계층)으로 이뤄진 네트워크의 예<br>
![image.png](https://i.imgur.com/28Q4Bnq.png)

위의 그림과 같이 완전연결 신경망은 Affine 계층 뒤에 활성화 함수를 갖는 ReLU 계층(혹은 Sigmoid 계층)이 이어집니다. 이 그림에서는 Affine-ReLU 조합이 4개가 쌓였고, 마지막 5번째 층은 Affine 계층에 이어 소프트맥스 계층에서  최종 결과(확률)를 출력합니다.

* CNN으로 이뤄진 네트워크의 예 : 합성곱 계층과 풀링 계층이 새로 추가(회색)<br>
![image.png](https://i.imgur.com/oUptKaL.png)

위의 그림과 같이 CNN에서는 새로운 '합성곱 계층(Conv)'과 '풀링 계층(Pooling)'이 추가됩니다. CNN의 계층은 'Conv-ReLU-(Pooling)' 흐름으로 연결됩니다(풀링 계층은 생략하기도 합니다). 지금까지의 'Affine-ReLU' 연결이 'Conv-ReLU-(Pooling)'으로 바뀌었다고 생각할 수 있겠죠.<br>
위의 그림의 CNN에서 주목할 또 다른 점은 출력에 가까운 층에서는 지금까지의 'Affine-ReLU' 구성을 사용할 수 있다는 것입니다. 또, 마지막 출력 계층에서는 'Affine-Softmax' 조합을 그대로 사용합니다.

### 완전연결 계층의 문제점(p229)

지금까지 본 완전연결 신경망에서는 완전연결 계층(Affine 계층)을 사용했습니다. 완전연결 계층에서는 인접하는 계층의 뉴런이 모두 연결되고 출력의 수는 임의로 정할 수 있습니다.

완전연결 계층의 문제점은 무엇일까요? 바로 '데이터의 형상이 무시'된다는 사실입니다. 입력 데이터가 이미지인 경우를 예로 들면, 이미지는 통상 세로/가로/채널(색상)로 구성된 3차원 데이터입니다. 그러나 완전연결 계층에 입력할 때는 3차원 데이터를 평평한 1차원 데이터로 평탄화해줘야 합니다. 사실 지금까지의 MNIST 데이터셋을 사용한 사례에서는 형상이 (1,28,28)인 이미지(1채널, 세로 28픽셀, 가로 28픽셀)를 1줄로 세운 784개의 데이터를 첫 Affine 계층에 입력했습니다.

이미지는 3차원 형상이며, 이 형상에는 소중한 공간적 정보가 담겨 있죠. 예를 들어 공간적으로 가까운 픽셀은 값이 비슷하거나, RGB의 각 채널은 서로 밀접하게 관련되어 있거나, 거리가 먼 픽셀끼리는 별 연관이 없는 등, 3차원 속에서 의미를 갖는 본질적인 패턴이 숨어 있을 것입니다. 그러나 완전연결 계층은 형상을 무시하고 모든 입력 데이터를 동등한 뉴런(같은 차원의 뉴런)으로 취급하여 형상에 담긴 정보를 살릴 수 없습니다.

한편, 합성곱 계층은 형상을 유지합니다. 이미지도 3차원 데이터로 입력받으며, 마찬가지로 다음 계층에도 3차원 데이터로 전달합니다. 그래서 CNN에서는 이미지처럼 형상을 가진 데이터를 제대로 이해할 (가능성이 있는) 것입니다.

### 합성곱 연산(p230)

* 합성곱 연산의 계산 순서<br>
![image.png](https://i.imgur.com/pypW5oW.png)

자, 완전연결 신경망에는 가중치 매개변수와 편향이 존재하는데, CNN에서는 필터의 매개변수가 그동안의 '가중치'에 해당합니다. 그리고 CNN에도 편향이 존재합니다. 위의 그림은 필터를 적용하는 단계까지만 보여준 것이고, 편향까지 포함하면 아래 그림과 같은 흐름이 됩니다.

* 합성곱 연산의 편향 : 필터를 적용할 원소에 고정값(편향)을 더한다.<br>
![image.png](https://i.imgur.com/WpBkcwJ.png)

위의 그림과 같인 편향은 필터를 적용한 후의 데이터에 더해집니다. 그리고 편향은 항상 하나(1x1)만 존재합니다. 그 하나의 값을 필터를 적용한 모든 원소에 더하는 것이죠.

### 패딩(p232)

합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값(예컨대 0)으로 채우기도 합니다. 이를 패딩(padding)이라 하며, 합성곱 연산에서 자주 이용하는 기법입니다.

* 합성곱 연산의 패딩 처리 : 입력 데이터 주위에 0을 채운다(패딩은 점선으로 표시했으며 그 안의 값 '0'은 생략했다).<br>
![image.png](https://i.imgur.com/GtA90VS.png)

패딩은 주로 출력 크기를 조정할 목적으로 사용합니다. 예를 들어 (4,4) 입력 데이터에 (3,3) 필터를 적용하면 출력은 (2,2)가 되어, 입력보다 2만큼 줄어듭니다. 이는 합성곱 연산을 몇 번이나 되풀이하는 심층 신경망에서 문제가 될 수 있습니다. 합성곱 연산을 거칠 때마다 크기가 작아지면 어느 시점에서는 출력 크기가 1이 되어버리겠죠. 더 이상은 합성곱 연산을 적용할 수 없다는 뜻입니다. 이러한 사태를 막기 위해 패딩을 사용합니다. 앞의 예에서는 패딩의 폭을 1로 설정하니 (4,4) 입력에 대한 출력이 같은 크기인 (4,4)로 유지되었습니다. 한 마디로 입력 데이터의 공간적 크기를 고정한 채로 다음 계층에 전달할 수 있습니다.

### 스트라이드(p233)

* 스트라이드가 2인 합성곱 연산<br>
![image.png](https://i.imgur.com/C6AcvXB.png)

위의 그림에서는 크기가 (7,7)인 입력 데이터에 스트라이드를 2로 설정한 필터를 적용합니다. 이처럼 스트라이드는 필터를 적용하는 간격을 지정합니다.

입력 크기를 (H,W), 필터 크기를 (FH, FW), 출력 크기를(OH, OW), 패딩을 P, 스트라이드를 S라 하면, 출력 크기는 다음 식으로 계산합니다.

![image.png](https://i.imgur.com/ak9vkUD.png)

### 3차원 데이터의 합성곱 연산(p235)

* 3차원 데이터 합성곱 연산의 계산 순서<br>
![image.png](https://i.imgur.com/sDZYsFh.png)

3차원의 합성곱 연산에서 주의할 점은 입력 데이터의 채널 수와 필터의 채널 수가 같아야 한다는 것입니다. 이 예에서는 모두 3개로 일치합니다. 한편, 필터 자체의 크기는 원하는 값으로 설정할 수 있습니다(단, 모든 채널의 필터가 같은 크기여야 합니다). 이 예에서는 (3,3)이지만, 원한다면 (2,2)나 (1,1) 또는 (5,5) 등으로 설정해도 되는 것이죠. 다시 말하지만, 필터의 채널 수는 입력 데이터의 채널 수와 같도록(이 예에서는 3) 설정해야 합니다.

### 블록으로 생각하기(p237)

3차원의 합성곱 연산은 데이터와 필터를 직육면체 블록이라고 생각하면 쉽습니다. 또, 3차원 데이터를 다차원 배열로 나타낼 때는 (채널(channel), 높이(height), 너비(width)) 순서로 쓰겠습니다. 예를 들어 채널 수 C, 높이 H, 너비 W인 데이터의 형상은 (C, H, W)로 씁니다. 필터도 같은 순서로 씁니다. 예를 들어 채널 수 C, 필터 높이 FH(Filter Height), 필터 너비 FW(Filter Width)의 경우 (C, FH, FW)로 씁니다.

* 합성곱 연산을 직육면체 블록으로 생각한다. 블록의 형상에 유의할 것!<br>
![image.png](https://i.imgur.com/xGh1UC6.png)

자, 이 예에서 출력 데이터는 한 장의 특징 맵입니다. 한 장의 특징 맵을 다른 말로 하면 채널이 1개인 특징 맵이죠. 그럼 합성곱 연산의 출력으로 다수의 채널을 내보내려면 어떻게 해야 할까요? 그 답은 필터(가중치)를 다수 사용하는 것입니다. 그림으로는 아래처럼 됩니다.

* 여러 필터를 사용한 합성곱 연산의 예<br>
![image.png](https://i.imgur.com/XE1vnhz.png)

이 그림과 같이 필터를 FN개 적용하면 출력 맵도 FN개가 생성됩니다. 그리고 그 FN개의 맵을 모으면 형상이 (FN, OH, OW)인 블록이 완성됩니다. 이 완성된 블록을 다음 계층으로 넘기겠다는 것이 CNN의 처리 흐름입니다.

이상에서 보듯 합성곱 연산에서는 필터의 수도 고려해야 합니다. 그런 이유로 필터의 가중치 데이터는 4차원 데이터이며 (출력 채널 수, 입력 채널 수, 높이, 너비) 순으로 씁니다. 예를 들어 채널 수 3, 크기 5x5인 필터가 20개 있다면 (20,3,5,5)로 씁니다.

자, 합성곱 연산에도 (완전연결 계층과 마찬가지로) 편향이 쓰입니다. 아래 그림은 편향을 더한 모습입니다.

![image.png](https://i.imgur.com/mN6GGfm.png)

위의 그림에서 보듯 편향은 채널 하나에 값 하나씩으로 구성됩니다. 이 예에서는 편향의 형상은 (FN,1,1)이고 ,필터의 출력 결과의 형상은 (FN, OH, OW)입니다. 이 두 블록을 더하면 편향의 각 값이 필터의 출력인 (FN, OH, OW) 블록의 대응 채널의 원소 모두에 더해집니다. 참고로, 형상이 다른 블록의 덧셈은 넘파이의 브로드캐스트 기능으로 쉽게 구현할 수 있습니다.

### 배치 처리(p239)

신경망 처리에서는 입력 데이터를 한 덩어리로 묶어 배치로 처리했습니다. 완전연결 신경망을 구현하면서는 이 방식을 지원하여 처리 효율을 높이고, 미니배치 방식의 학습도 지원하도록 했습니다.<br>
합성곱 연산도 마찬가지로 배치 처리를 지원하고자 합니다. 그래서 각 계층을 흐르는 데이터의 차원을 하나 늘려 4차원 데이터로 저장합니다. 구체적으로는 데이터를 (데이터 수, 채널 수, 높이, 너비) 순으로 저장합니다. 데이터가 N개일 때 위의 그림을 배치 처리한다면 데이터 형태가 아래 그림처럼 되는 것이죠.

* 합성곱 연산의 처리 흐름(배치 처리)<br>
![image.png](https://i.imgur.com/WLtCDlQ.png)

배치 처리 시의 데이터 흐름을 나타낸 위의 그림을 보면 각 데이터의 선두에 배치용 차원을 추가했습니다. 이처럼 데이터는 4차원 형상을 가진 채 각 계층을 타고 흐릅니다. 여기에서 주의할 점으로는 신경망에 4차원 데이터가 하나 흐를 때마다 데이터 N개에 대한 합성곱 연산이 이뤄진다는 것입니다. 즉, N회 분의 처리를 한 번에 수행하는 것이죠.

### 풀링 계층(p240)